# Pandas CSV Reading Performance Tips

In [ ]:
import pandas as pd

# ## 1. Use dtype to specify column types
# Avoid pandas guessing data types, which is slower and less efficient.
# Define dtypes for each column
dtypes = {
    'id': 'int32',
    'name': 'category',
    'amount': 'float32',
    'status': 'category'
}
df = pd.read_csv('data.csv', dtype=dtypes)

In [ ]:
# ## 2. Use usecols to load only required columns
df = pd.read_csv('data.csv', usecols=['id', 'amount'])

In [ ]:
# ## 3. Parse dates efficiently using parse_dates
df = pd.read_csv('data.csv', parse_dates=['created_at'])

In [ ]:
# ## 4. Use low_memory=False when you get mixed types warnings
# This prevents pandas from internally chunking and guessing dtypes.
# Better alternative: use explicit dtypes.
df = pd.read_csv('data.csv', low_memory=False)

In [ ]:
# ## 5. Process large files in chunks using chunksize
chunks = pd.read_csv('data.csv', chunksize=100_000)
for chunk in chunks:
    process(chunk)  # Replace with your processing logic

In [ ]:
# ## 6. Use compression if reading from zipped files
df = pd.read_csv('data.csv.gz', compression='gzip')

In [ ]:
# ## 7. Disable quoting if not needed (speeds up parsing)
import csv
df = pd.read_csv('data.csv', quoting=csv.QUOTE_NONE)

In [ ]:
# ## 8. Use faster backend engines (pandas 2.0+)
df = pd.read_csv('data.csv', engine='pyarrow')  # Alternative to engine='c'